In [21]:
from trajectree.fock_optics.utils import create_vacuum_state
from trajectree.fock_optics.devices import global_phase
from trajectree.quant_info.noise_models import amplitude_damping
from trajectree.quant_info.circuit import Circuit
from trajectree.trajectory import quantum_channel, trajectory_evaluator
import numpy as np
from scipy import sparse as sp
from matplotlib import pyplot as plt
import cirq
import qsimcirq
import time

from qiskit.quantum_info import SparsePauliOp
from qiskit.transpiler import generate_preset_pass_manager
from qiskit_aer import AerSimulator
from qiskit_aer.primitives import EstimatorV2 as Estimator
from qiskit import transpile, QuantumCircuit
from qiskit_aer.primitives import Sampler as AerSampler
from qiskit.primitives import SamplerResult
from qiskit_aer.noise import (
    NoiseModel,
    QuantumError,
    ReadoutError,
    depolarizing_error,
    amplitude_damping_error,
    thermal_relaxation_error,
)

from mqt.bench import BenchmarkLevel, get_benchmark

Benchmarking circuits

In [22]:
num_qubits = 20
num_trajectories = 1
qc = get_benchmark("dj", BenchmarkLevel.ALG, num_qubits)
qc.draw(output="mpl")


# from qiskit import QuantumCircuit
 
# qc = QuantumCircuit(num_qubits)
# for i in range(num_qubits):
#     qc.x(i)
# for i in range(num_qubits):
#     qc.h(i)
# for i in range(1, num_qubits):
#     qc.cx(i-1, i)
qc.draw()

┌───┐     ┌──────────┐┌───┐ ░ ┌─┐                                       »
 q_0: ┤ H ├─────┤0         ├┤ H ├─░─┤M├───────────────────────────────────────»
      ├───┤     │          │├───┤ ░ └╥┘┌─┐                                    »
 q_1: ┤ H ├─────┤1         ├┤ H ├─░──╫─┤M├────────────────────────────────────»
      ├───┤     │          │├───┤ ░  ║ └╥┘┌─┐                                 »
 q_2: ┤ H ├─────┤2         ├┤ H ├─░──╫──╫─┤M├─────────────────────────────────»
      ├───┤     │          │├───┤ ░  ║  ║ └╥┘┌─┐                              »
 q_3: ┤ H ├─────┤3         ├┤ H ├─░──╫──╫──╫─┤M├──────────────────────────────»
      ├───┤     │          │├───┤ ░  ║  ║  ║ └╥┘┌─┐                           »
 q_4: ┤ H ├─────┤4         ├┤ H ├─░──╫──╫──╫──╫─┤M├───────────────────────────»
      ├───┤     │          │├───┤ ░  ║  ║  ║  ║ └╥┘┌─┐                        »
 q_5: ┤ H ├─────┤5         ├┤ H ├─░──╫──╫──╫──╫──╫─┤M├────────────────────────»
      ├───┤     │          │├───┤ ░  ║  ║  ║  ║  ║ └╥┘┌─┐                     »
 q_6: ┤ H ├─────┤6         ├┤ H ├─░──╫──╫──╫──╫──╫──╫─┤M├─────────────────────»
      ├───┤     │          │├───┤ ░  ║  ║  ║  ║  ║  ║ └╥┘┌─┐                  »
 q_7: ┤ H ├─────┤7         ├┤ H ├─░──╫──╫──╫──╫──╫──╫──╫─┤M├──────────────────»
      ├───┤     │          │├───┤ ░  ║  ║  ║  ║  ║  ║  ║ └╥┘┌─┐               »
 q_8: ┤ H ├─────┤8         ├┤ H ├─░──╫──╫──╫──╫──╫──╫──╫──╫─┤M├───────────────»
      ├───┤     │          │├───┤ ░  ║  ║  ║  ║  ║  ║  ║  ║ └╥┘┌─┐            »
 q_9: ┤ H ├─────┤9         ├┤ H ├─░──╫──╫──╫──╫──╫──╫──╫──╫──╫─┤M├────────────»
      ├───┤     │   Oracle │├───┤ ░  ║  ║  ║  ║  ║  ║  ║  ║  ║ └╥┘┌─┐         »
q_10: ┤ H ├─────┤10        ├┤ H ├─░──╫──╫──╫──╫──╫──╫──╫──╫──╫──╫─┤M├─────────»
      ├───┤     │          │├───┤ ░  ║  ║  ║  ║  ║  ║  ║  ║  ║  ║ └╥┘┌─┐      »
q_11: ┤ H ├─────┤11        ├┤ H ├─░──╫──╫──╫──╫──╫──╫──╫──╫──╫──╫──╫─┤M├──────»
      ├───┤     │          │├───┤ ░  ║  ║  ║  ║  ║  ║  ║  ║  ║  ║  ║ └╥┘┌─┐   »
q_12: ┤ H ├─────┤12        ├┤ H ├─░──╫──╫──╫──╫──╫──╫──╫──╫──╫──╫──╫──╫─┤M├───»
      ├───┤     │          │├───┤ ░  ║  ║  ║  ║  ║  ║  ║  ║  ║  ║  ║  ║ └╥┘┌─┐»
q_13: ┤ H ├─────┤13        ├┤ H ├─░──╫──╫──╫──╫──╫──╫──╫──╫──╫──╫──╫──╫──╫─┤M├»
      ├───┤     │          │├───┤ ░  ║  ║  ║  ║  ║  ║  ║  ║  ║  ║  ║  ║  ║ └╥┘»
q_14: ┤ H ├─────┤14        ├┤ H ├─░──╫──╫──╫──╫──╫──╫──╫──╫──╫──╫──╫──╫──╫──╫─»
      ├───┤     │          │├───┤ ░  ║  ║  ║  ║  ║  ║  ║  ║  ║  ║  ║  ║  ║  ║ »
q_15: ┤ H ├─────┤15        ├┤ H ├─░──╫──╫──╫──╫──╫──╫──╫──╫──╫──╫──╫──╫──╫──╫─»
      ├───┤     │          │├───┤ ░  ║  ║  ║  ║  ║  ║  ║  ║  ║  ║  ║  ║  ║  ║ »
q_16: ┤ H ├─────┤16        ├┤ H ├─░──╫──╫──╫──╫──╫──╫──╫──╫──╫──╫──╫──╫──╫──╫─»
      ├───┤     │          │├───┤ ░  ║  ║  ║  ║  ║  ║  ║  ║  ║  ║  ║  ║  ║  ║ »
q_17: ┤ H ├─────┤17        ├┤ H ├─░──╫──╫──╫──╫──╫──╫──╫──╫──╫──╫──╫──╫──╫──╫─»
      ├───┤     │          │├───┤ ░  ║  ║  ║  ║  ║  ║  ║  ║  ║  ║  ║  ║  ║  ║ »
q_18: ┤ H ├─────┤18        ├┤ H ├─░──╫──╫──╫──╫──╫──╫──╫──╫──╫──╫──╫──╫──╫──╫─»
      ├───┤┌───┐│          │└───┘ ░  ║  ║  ║  ║  ║  ║  ║  ║  ║  ║  ║  ║  ║  ║ »
q_19: ┤ X ├┤ H ├┤19        ├──────░──╫──╫──╫──╫──╫──╫──╫──╫──╫──╫──╫──╫──╫──╫─»
      └───┘└───┘└──────────┘      ░  ║  ║  ║  ║  ║  ║  ║  ║  ║  ║  ║  ║  ║  ║ »
c: 19/═══════════════════════════════╩══╩══╩══╩══╩══╩══╩══╩══╩══╩══╩══╩══╩══╩═»
                                     0  1  2  3  4  5  6  7  8  9  10 11 12 13»
«                     
« q_0: ───────────────
«                     
« q_1: ───────────────
«                     
« q_2: ───────────────
«                     
« q_3: ───────────────
«                     
« q_4: ───────────────
«                     
« q_5: ───────────────
«                     
« q_6: ───────────────
«                     
« q_7: ───────────────
«                     
« q_8: ───────────────
«                     
« q_9: ───────────────
«                     
«q_10: ───────────────
«                     
«q_11: ───────────────
«             

In [23]:
# print(dumps(qc))
target_basis = ['h', 's', 'x', 't', 'cx']
transpiled_qc_custom = transpile(qc, basis_gates=target_basis, optimization_level=1)

noise_probability = 0.01
noise_channel = amplitude_damping_error(noise_probability)

two_qubit_noise_channel = noise_channel.tensor(noise_channel)

noise_model = NoiseModel()
noise_model.add_all_qubit_quantum_error(noise_channel, target_basis[:-1])
noise_model.add_all_qubit_quantum_error(two_qubit_noise_channel, ["cx"])

print(noise_model)
transpiled_qc_custom.draw(output="text")
# noisy_transpiled_qc_custom = QuantumCircuit(transpiled_qc_custom.num_qubits, transpiled_qc_custom.num_clbits)
# for instr, qargs, cargs in transpiled_qc_custom.data:
#     if instr.name not in {"measure", "barrier"}:
#         noisy_transpiled_qc_custom.append(instr, qargs, cargs)
#         for q in qargs:
#             noisy_transpiled_qc_custom.append(noise_channel, [q])        

# transpiled_qc_custom = noisy_transpiled_qc_custom

# transpiled_qc_custom.draw(output="mpl")

# # for i in transpiled_qc_custom:
# #     print(i.operation.name)
# #     print([qubit._index for qubit in i.qubits])
# # transpiled_qc_custom.draw(output="mpl")

NoiseModel:
  Basis gates: ['cx', 'h', 'id', 'rz', 's', 'sx', 't', 'x']
  Instructions with noise: ['x', 'h', 'cx', 't', 's']
  All-qubits errors: ['h', 's', 'x', 't', 'cx']


┌───┐┌───┐┌───┐     ┌───┐┌───┐                                        »
 q_0: ┤ S ├┤ S ├┤ H ├──■──┤ X ├┤ H ├────────────────────────────────────────»
      ├───┤├───┤├───┤  │  └───┘├───┤┌───┐                                   »
 q_1: ┤ S ├┤ S ├┤ H ├──┼────■──┤ X ├┤ H ├───────────────────────────────────»
      ├───┤└───┘└───┘  │    │  └───┘├───┤                                   »
 q_2: ┤ H ├────────────┼────┼────■──┤ H ├───────────────────────────────────»
      ├───┤            │    │    │  └───┘┌───┐                              »
 q_3: ┤ H ├────────────┼────┼────┼────■──┤ H ├──────────────────────────────»
      ├───┤┌───┐┌───┐  │    │    │    │  └───┘┌───┐┌───┐                    »
 q_4: ┤ S ├┤ S ├┤ H ├──┼────┼────┼────┼────■──┤ X ├┤ H ├────────────────────»
      ├───┤├───┤├───┤  │    │    │    │    │  └───┘├───┤┌───┐               »
 q_5: ┤ S ├┤ S ├┤ H ├──┼────┼────┼────┼────┼────■──┤ X ├┤ H ├───────────────»
      ├───┤├───┤├───┤  │    │    │    │    │    │  └───┘├───┤┌───┐          »
 q_6: ┤ S ├┤ S ├┤ H ├──┼────┼────┼────┼────┼────┼────■──┤ X ├┤ H ├──────────»
      ├───┤└───┘└───┘  │    │    │    │    │    │    │  └───┘├───┤          »
 q_7: ┤ H ├────────────┼────┼────┼────┼────┼────┼────┼────■──┤ H ├──────────»
      ├───┤┌───┐┌───┐  │    │    │    │    │    │    │    │  └───┘┌───┐┌───┐»
 q_8: ┤ S ├┤ S ├┤ H ├──┼────┼────┼────┼────┼────┼────┼────┼────■──┤ X ├┤ H ├»
      ├───┤├───┤├───┤  │    │    │    │    │    │    │    │    │  └───┘├───┤»
 q_9: ┤ S ├┤ S ├┤ H ├──┼────┼────┼────┼────┼────┼────┼────┼────┼────■──┤ X ├»
      ├───┤└───┘└───┘  │    │    │    │    │    │    │    │    │    │  └───┘»
q_10: ┤ H ├────────────┼────┼────┼────┼────┼────┼────┼────┼────┼────┼────■──»
      ├───┤            │    │    │    │    │    │    │    │    │    │    │  »
q_11: ┤ H ├────────────┼────┼────┼────┼────┼────┼────┼────┼────┼────┼────┼──»
      ├───┤            │    │    │    │    │    │    │    │    │    │    │  »
q_12: ┤ H ├────────────┼────┼────┼────┼────┼────┼────┼────┼────┼────┼────┼──»
      ├───┤┌───┐┌───┐  │    │    │    │    │    │    │    │    │    │    │  »
q_13: ┤ S ├┤ S ├┤ H ├──┼────┼────┼────┼────┼────┼────┼────┼────┼────┼────┼──»
      ├───┤└───┘└───┘  │    │    │    │    │    │    │    │    │    │    │  »
q_14: ┤ H ├────────────┼────┼────┼────┼────┼────┼────┼────┼────┼────┼────┼──»
      ├───┤┌───┐┌───┐  │    │    │    │    │    │    │    │    │    │    │  »
q_15: ┤ S ├┤ S ├┤ H ├──┼────┼────┼────┼────┼────┼────┼────┼────┼────┼────┼──»
      ├───┤└───┘└───┘  │    │    │    │    │    │    │    │    │    │    │  »
q_16: ┤ H ├────────────┼────┼────┼────┼────┼────┼────┼────┼────┼────┼────┼──»
      ├───┤            │    │    │    │    │    │    │    │    │    │    │  »
q_17: ┤ H ├────────────┼────┼────┼────┼────┼────┼────┼────┼────┼────┼────┼──»
      ├───┤┌───┐┌───┐  │    │    │    │    │    │    │    │    │    │    │  »
q_18: ┤ S ├┤ S ├┤ H ├──┼────┼────┼────┼────┼────┼────┼────┼────┼────┼────┼──»
      ├───┤├───┤└───┘┌─┴─┐┌─┴─┐┌─┴─┐┌─┴─┐┌─┴─┐┌─┴─┐┌─┴─┐┌─┴─┐┌─┴─┐┌─┴─┐┌─┴─┐»
q_19: ┤ X ├┤ H ├─────┤ X ├┤ X ├┤ X ├┤ X ├┤ X ├┤ X ├┤ X ├┤ X ├┤ X ├┤ X ├┤ X ├»
      └───┘└───┘     └───┘└───┘└───┘└───┘└───┘└───┘└───┘└───┘└───┘└───┘└───┘»
c: 19/══════════════════════════════════════════════════════════════════════»
                                                                            »
«                                                         ░ ┌─┐               »
« q_0: ───────────────────────────────────────────────────░─┤M├───────────────»
«                                                         ░ └╥┘┌─┐            »
« q_1: ───────────────────────────────────────────────────░──╫─┤M├────────────»
«                                                         ░  ║ └╥┘┌─┐         »
« q_2: ───────────────────────────────────────────────────░──╫──╫─┤M├─────────»
«                                                         ░  ║  ║ └╥┘┌─┐      »
« q_3: ───────────────────────────────────────────────────░──╫──╫──╫─┤M├──────»
«           

In [24]:
observable = SparsePauliOp("Z" * (num_qubits))

# Use the Sampler primitive for latest API
backend = AerSampler()

noisy_estimator = Estimator(
    options=dict(backend_options=dict(noise_model=noise_model), run_options=dict(shots=num_trajectories))
)

pub = (transpiled_qc_custom, observable)

job = noisy_estimator.run([pub])
result = job.result()
pub_result = result[0]
# print((len(result)))
print(float(pub_result.data.evs))
print(float(pub_result.data.stds))


# estimator = Estimator()
# estimator.options.run_options = {"shots":10000}
# estimator.options.backend_options = dict(noise_model=noise_channel)
# # print(estimator.options)

# # Run the sampler
# job = estimator.run([(transpiled_qc_custom, observable)])
# result = job.result()

# pub_result = result[0]
# print(result)
# print(pub_result.data.stds)
# print(pub_result.data.evs)

0.005037938313141144
0.0


In [34]:
import cirq
import os
os.environ["OMP_NUM_THREADS"] = "1"
os.environ["MKL_NUM_THREADS"] = "1"
os.environ["OPENBLAS_NUM_THREADS"] = "1"


# Create Cirq qubits
num_qubits_cirq = transpiled_qc_custom.num_qubits
cirq_qubits = cirq.LineQubit.range(num_qubits_cirq)

# Map Qiskit gates to Cirq gates
gate_map = {
    'h': lambda q: cirq.H(q),
    's': lambda q: cirq.S(q),
    't': lambda q: cirq.T(q),
    'x': lambda q: cirq.X(q),
    'cx': lambda q0, q1: cirq.CNOT(q0, q1),
    'measure': lambda q: None,  # We'll handle measurements separately
    'barrier': lambda *args: None,
}

# Build the Cirq circuit
cirq_circuit = cirq.Circuit()

for circuit_instr in transpiled_qc_custom.data:
    instr = circuit_instr.operation
    qargs = circuit_instr.qubits
    cargs = circuit_instr.clbits
    gate_name = instr.name
    
    # if gate_name == 'quantum_channel':
    #     # This is the amplitude damping error
    #     # Extract the damping parameter from the Kraus operators
    #     for qarg in qargs:
    #         cirq_qubit = cirq_qubits[qarg._index]
    #         cirq_circuit.append(cirq.amplitude_damp(gamma=noise_probability)(cirq_qubit))
    
    if gate_name in gate_map and gate_name not in ['measure', 'barrier']:
        cirq_qubit_args = [cirq_qubits[q._index] for q in qargs]
        gate = gate_map[gate_name](*cirq_qubit_args)
        
        if gate is not None:
            cirq_circuit.append(gate)
            for qarg in qargs:
                cirq_qubit = cirq_qubits[qarg._index]
                cirq_circuit.append(cirq.amplitude_damp(gamma=noise_probability)(cirq_qubit))

# # Add measurements at the end
# cirq_circuit.append(cirq.measure(*cirq_qubits, key='result'))

print(f"Converted Cirq circuit with {len(cirq_circuit)} moments")
print(f"Number of operations: {len(list(cirq_circuit.all_operations()))}")
print("\nFirst 50 operations:")
for i, op in enumerate(cirq_circuit.all_operations()):
    if i >= 50:
        break
    print(f"{i}: {op}")

# Draw a portion of the circuit
print("\nCircuit diagram (first 20 moments):")
print(cirq.Circuit(list(cirq_circuit)))


Converted Cirq circuit with 48 moments
Number of operations: 197

First 50 operations:
0: S(q(0))
1: S(q(1))
2: H(q(2))
3: H(q(3))
4: S(q(4))
5: S(q(5))
6: S(q(6))
7: H(q(7))
8: S(q(8))
9: S(q(9))
10: H(q(10))
11: H(q(11))
12: H(q(12))
13: S(q(13))
14: H(q(14))
15: S(q(15))
16: H(q(16))
17: H(q(17))
18: S(q(18))
19: X(q(19))
20: amplitude_damp(gamma=0.01)(q(0))
21: amplitude_damp(gamma=0.01)(q(1))
22: amplitude_damp(gamma=0.01)(q(2))
23: amplitude_damp(gamma=0.01)(q(3))
24: amplitude_damp(gamma=0.01)(q(4))
25: amplitude_damp(gamma=0.01)(q(5))
26: amplitude_damp(gamma=0.01)(q(6))
27: amplitude_damp(gamma=0.01)(q(7))
28: amplitude_damp(gamma=0.01)(q(8))
29: amplitude_damp(gamma=0.01)(q(9))
30: amplitude_damp(gamma=0.01)(q(10))
31: amplitude_damp(gamma=0.01)(q(11))
32: amplitude_damp(gamma=0.01)(q(12))
33: amplitude_damp(gamma=0.01)(q(13))
34: amplitude_damp(gamma=0.01)(q(14))
35: amplitude_damp(gamma=0.01)(q(15))
36: amplitude_damp(gamma=0.01)(q(16))
37: amplitude_damp(gamma=0.01)(q(17))

In [33]:
# Set the "noisy repetitions" to 100.
# This parameter only affects expectation value calculations.
options = {'r': num_trajectories}
# Also set the random seed to get reproducible results.
ev_simulator = qsimcirq.QSimSimulator(qsim_options=options)
# Define observables to measure: <Z> for q0 and <X> for q1.
observable = cirq.Z(cirq_qubits[0])
for i in range(1,num_qubits):
    observable *= cirq.Z(cirq_qubits[i])
# Calculate expectation values for the given observables.
start_time = time.time()
ev_results = ev_simulator.simulate_expectation_values(
    cirq_circuit,
    observables=[observable],
)
print("elapsed time:", time.time() - start_time)
print(ev_results)

elapsed time: 0.05258369445800781
[(2.529134490545596e-05+0j)]


## Now, we implement the same circuit in Trajectree

In [27]:
qc = Circuit(num_qubits)
qc.qiskit_to_trajectree(transpiled_qc_custom, noise_parameter=noise_probability)
# qc.expectation('z'*(num_qubits))

evs, times = qc.perform_trajectree_simulation(num_trajectories)
print("outside of simulation")
# np.mean(evs)
# evs

time taken: 11.251485824584961
done with simulations
outside of simulation


In [ ]:
print(len(qc.t_eval.quantum_channels))

110


In [ ]:
qc = Circuit(3)
qc.H_gate(0)
qc.CNOT_gate(0, 2)
qc.create_trajectree()
qc.expectation('xxz')
qc.perform_trajectree_simulation(1)[0][0]

np.complex128(0j)

In [ ]:
def run_experiment(max_cache_nodes=-1):
    num_simulations = 100 # 20

    num_qubits = 10

    cache_size = 1
    iter = 0
    max_iter = 1
    times = []

    while iter < max_iter:  # This while loop is just to perform the entire exoeriment multiple times to get average runtime values.
        # psi, t_eval = generate_test_circuit_trajectree(cache_size, max_cache_nodes)
        qc = Circuit(num_qubits)
        
        for i in range(num_qubits):
            qc.H_gate(i)

        # Damping layer
        for i in range(num_qubits):
            qc.amplitude_damping(0.1, i)

        qc.create_trajectree()

        times_iter = qc.perform_trajectree_simulation(num_simulations)

        times.append(times_iter)
        
        iter += 1
        print("iter:", iter)

    times_avg = np.mean(np.array(times).T, axis = 1)

    avg_times = [np.mean(times_avg[:i]) for i in range(1, len(times_avg))]

    return qc
    # return times
run_experiment()

iter: 1


AttributeError: 'float' object has no attribute 'site_tags'